In [1]:
#importing necessary libraries 
import requests 
import csv
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as p
import folium
import polyline
import os

from datetime import datetime
from folium.plugins import MarkerCluster
%matplotlib inline

Below I used the Alberta government's public API to get the real time data about conditions around the provience. Since the data was in json format, I convered it to CSV format and saved it as 'weather.csv' file. 

In [2]:
# api_address_url = 'https://511.alberta.ca/api/v2/get/winterroads'
# json_data = requests.get(api_address_url).json()

# weather_data = open('weather.csv', 'w')
# csvwriter = csv.writer(weather_data)
# count = 0

# for emp in json_data:

#     if count == 0:

#         header = emp.keys()

#         csvwriter.writerow(header)

#         count += 1

#     csvwriter.writerow(emp.values())

# weather_data.close()

# EDA

I will be answering following questions in these notebook as well share insights as I go along. 

1] What is the percentage of locations with good visibility?\
2] Which areas of the province are experiencing poor visibility? \
3] Which areas have bad conditions?\
4] Can you provide a visualization showcasing the different conditions experienced by different parts of the province? \ 
\
BONUS: What other interesting insights can you find? How can this be used by the government to improve people's wellbeing?


In [3]:
data = pd.read_csv('weather.csv')
data.head()

,LocationDescription,Primary Condition,Secondary Conditions,Visibility,AreaName,RoadwayName,EncodedPolyline,LastUpdated
0,Chief Mountain Customs to Belly River Gate,Closed,[],NaN,WATERTON PARKS,Hwy 6,qz`jH|jvtTw@j@??yA|@??mBhAqCdBo@\u@\{@\uAd@}E|...,1571943579
1,Belly River Gate to Two Flags Viewpoint,Closed,[],NaN,WATERTON PARKS,Hwy 6,qpjjHlv{tTQjCQlC??MlB??WtDM~AK~@??Kn@Ol@Sn@Uh@...,1571943583
2,Moraine Lake to Lake Louise Drive,Closed,[],NaN,BANFF,Moraine Lake Rd,{lhxHjqbdUi@e@{IcCeBkBa@k@a@UuDe@i@Bs@Pq@HcDw@...,1571943656
3,Lake Minnewanka to Johnson Lake,Closed,[],NaN,BANFF,Johnson Lake Road,m|nwH|q{_Uu_@`HkAFoGIy@F_Ah@q@|@YvAKnA@lAPfAd@...,1571943788
4,Jct Hwy 5 to Red Rock Canyon,Closed,[],NaN,WATERTON PARKS,RedRock Parkway,}lpjHn`avTq@~@_@xBWn@a@n@[\a@Pm@Ic@Y]WYc@Ki@Ae...,1571944252


In [4]:
print("The size of the dataset is: ",data.shape)

The size of the dataset is:  (565, 8)


Here we have 565 observations and 8 columns. Let's start with some data wrangling process. The 'EncodedPolyline' column is basically a string storing a series of coordinates and we can extract a series of latitudes and longitudes from it. By using the polyline library we can decode the string into coordinates. Just to demonstrate, let's decode the first row in the above data.

In [5]:
polyline.decode(data['EncodedPolyline'][0])[:5]

[(48.99769, -113.66079),
 (48.99797, -113.66101),
 (48.99797, -113.66101),
 (48.99842, -113.66132),
 (48.99842, -113.66132)]

So we get a series of coordinates, which I believe starts at Chief Mountain all the way to Belly River Gate. Since for question 4 we want to visualize the weather conditions, I decided to just keep the first tuple from the above list so for instance (48.99769, -113.66079) for first row/observation. I will loop through the entire 'EncodedPolyline' column in the code below to decode coordinates and store the latitude and longitude in new columns. 

In [6]:
latitude = []
longitude = []
for ind in data.index:
    encode = polyline.decode(data['EncodedPolyline'][ind])
    latitude.append(encode[0][0])
    longitude.append(encode[0][1])
    
data['latitude'] = latitude
data['longitude'] = longitude    
        
    

The 'LastUpdated' column is basically in unix time so we can extract datetime using the following code. 

In [7]:
# year = []
# minute = []
# hour = []
# month = []
# day = []

DateTime = []
for time in data['LastUpdated']:
    t = datetime.fromtimestamp(time)
    DateTime.append(t)
DateTime

data['DateTime'] = DateTime
#     year.append(t.year)
#     month.append(t.month)
#     day.append(t.day)
#     hour.append(t.hour)
#     minute.append(t.minute)
    
# data['year'] = year
# data['month'] = month
# data['day'] = day
# data['hour'] = hour
# data['minute'] = minute
    

We can now drop the original 'EncodedPolyline','LastUpdated'columns as we have stored the informations in a more understandable format.

In [8]:
df = data.drop(['EncodedPolyline','LastUpdated'], axis=1)
df.head()

,LocationDescription,Primary Condition,Secondary Conditions,Visibility,AreaName,RoadwayName,latitude,longitude,DateTime
0,Chief Mountain Customs to Belly River Gate,Closed,[],NaN,WATERTON PARKS,Hwy 6,48.99769,-113.66079,2019-10-24 13:59:39
1,Belly River Gate to Two Flags Viewpoint,Closed,[],NaN,WATERTON PARKS,Hwy 6,49.04729,-113.68823,2019-10-24 13:59:43
2,Moraine Lake to Lake Louise Drive,Closed,[],NaN,BANFF,Moraine Lake Rd,51.33022,-116.18086,2019-10-24 14:00:56
3,Lake Minnewanka to Johnson Lake,Closed,[],NaN,BANFF,Johnson Lake Road,51.19959,-115.48975,2019-10-24 14:03:08
4,Jct Hwy 5 to Red Rock Canyon,Closed,[],NaN,WATERTON PARKS,RedRock Parkway,49.07743,-113.87928,2019-10-24 14:10:52


In [9]:
print('Size of the new dataset: ',df.shape)

Size of the new dataset:  (565, 9)


Let's start EDA by checking how much of the data is missing.

In [10]:
round(df.isnull().sum()/len(df),2)*100

LocationDescription     0.0
Primary Condition       0.0
Secondary Conditions    0.0
Visibility              5.0
AreaName                0.0
RoadwayName             0.0
latitude                0.0
longitude               0.0
DateTime                0.0
dtype: float64

We see that 'Visibility' column has about 5% missing observations and from above we see that the 'Secondary Conditions' has lots of [ ] represented as empty list which I assume corresponds to road closed/closure. We can fill in the missing values in the 'Visibility' column after further inspection. For now we will just drop the missing data since it is just a small amount. 

In [11]:
df.dropna(inplace=True)
df.shape

(538, 9)

#  What is the percentage of locations with good visibility?

In [12]:
print("Percentage of locations with visibility = good:\n"+ str(df['Visibility'].value_counts()/len(df)*100))

Percentage of locations with visibility = good:
Good    99.814126
Fair     0.185874
Name: Visibility, dtype: float64


So about 99.8% of the locations have good visibility. In the cell below, uncomment below to print all the AreaName corresponding to 'fair' and 'good' visibility.  

In [13]:
visibility_area = df.groupby('Visibility').agg({'AreaName':'value_counts'})
visibility_area.columns = ['locations count']
#visibility_area 

# Which areas of the province are experiencing poor visibility? 

Since we only have 'good' and 'fair' in Visibility. I will show the information below where Visibility = 'Fair'.

In [14]:
df[df['Visibility']=='Fair']

,LocationDescription,Primary Condition,Secondary Conditions,Visibility,AreaName,RoadwayName,latitude,longitude,DateTime
355,Jct Hwy 724 to 276m east of Range Road 70,Bare Wet,"['Snowing', 'Shoulder Ice/Snow']",Fair,CMA 504,Hwy 43,55.17018,-119.15389,2019-11-01 17:54:07


So from Jct Hwy 724 to 276m east of range road 70, the visibility is fair due to snow. There is a chance of poor visibility in this route depending on the further weather forecast. The AreaName refers to Grande Prairie and Whitecourt, see link https://www.alberta.ca/highway-maintenance.aspx.

# Which areas have bad conditions?

In [15]:
df['Primary Condition'].value_counts()

Bare Dry        464
Bare Wet         65
Cvd Snw           3
Ptly Cvd Ice      2
Ptly Cvd Snw      2
Cvd Ice           2
Name: Primary Condition, dtype: int64

In [16]:
df['Secondary Conditions'].value_counts()

[]                                                   484
['Icy sections']                                      16
['Rain']                                              15
['Shoulder Ice/Snow']                                  8
['Snowing']                                            5
['Snowing', 'Shoulder Ice/Snow']                       4
['Shoulder Ice/Snow', 'Snowing']                       2
['Rain', 'Slush']                                      1
['Shoulder Ice/Snow', 'Snowing', 'Rain', 'Slush']      1
['Snowing', 'Shoulder Ice/Snow', 'Rain', 'Slush']      1
['Slush', 'Rain', 'Shoulder Ice/Snow', 'Snowing']      1
Name: Secondary Conditions, dtype: int64

We have 465 areas with Bare Dry conditions \
65 areas with bare wet conditions \
3 areas covered in snow \
2 areas partially covered in snow \
2 areas covered in ice 

There are high chances of road closure in the 2 areas thats covered in snow if the snow continuous, although visibility is good at present. It is advisable to not speed up or turn excessively, leave more space between you and other vehicles in the areas Cvd Snw/Ptly Cvd Snw.

The areas that's covered completely in ice or partly covered in ice, it is advisable to limit your speed and don't brake hard, equip you vehicles with good treads/traction for ice, avoid going uphill. 

We can even look at the RoadwayNames and LocationDescription to get more information for these specific conditions.

In [17]:
df[(df['Primary Condition'] == 'Cvd Snw') | (df['Primary Condition']=='Cvd Ice')| (df['Primary Condition']=='Ptly Cvd Ice')
  | (df['Primary Condition']=='Ptly Cvd Snw')]

,LocationDescription,Primary Condition,Secondary Conditions,Visibility,AreaName,RoadwayName,latitude,longitude,DateTime
37,Medicine Lake to Maligne Lake,Ptly Cvd Ice,['Icy sections'],Good,JASPER,Maligne Road,52.87309,-117.80593,2019-11-01 10:24:24
38,Maligne Canyon to Medicine Lake,Ptly Cvd Ice,['Icy sections'],Good,JASPER,Maligne Road,52.89416,-117.87477,2019-11-01 10:24:34
362,Cadomin Access to Teck Coal Mine Access Road,Cvd Ice,['Icy sections'],Good,CMA 508,Hwy 40,53.05314,-117.31731,2019-11-01 17:55:04
363,Coalspur to Cadomin Access,Cvd Ice,['Shoulder Ice/Snow'],Good,CMA 508,Hwy 40,53.17092,-117.00642,2019-11-01 17:55:07
509,Little Smoky to Fox Creek,Ptly Cvd Snw,"['Snowing', 'Shoulder Ice/Snow', 'Rain', 'Slush']",Good,CMA 03,Hwy 43,54.39422,-116.80913,2019-11-01 19:05:01
510,Fox Creek to Eagle Tower Road,Ptly Cvd Snw,"['Slush', 'Rain', 'Shoulder Ice/Snow', 'Snowing']",Good,CMA 03,Hwy 43,54.29738,-116.25832,2019-11-01 19:05:19
519,Jct Hwy 16 to Township Road 570,Cvd Snw,"['Shoulder Ice/Snow', 'Snowing', 'Rain', 'Slush']",Good,CMA 508,Hwy 32,53.89004,-115.90913,2019-11-01 19:57:09
558,Coalspur to Jct Hwy 16,Cvd Snw,['Shoulder Ice/Snow'],Good,CMA 508,Hwy 47,53.17165,-117.00728,2019-11-01 22:47:04
559,Lovett River Provincial Recreation Area to Coa...,Cvd Snw,['Shoulder Ice/Snow'],Good,CMA 508,Hwy 40,53.16813,-117.00254,2019-11-01 22:47:13


Below we can groupby the primary and secondary conditions. 

In [18]:
weather_conditons = df.groupby(['Primary Condition','Secondary Conditions']).agg({'AreaName':'value_counts'})
weather_conditons.columns = ['location counts']
#weather_conditons

# Can you provide a visualization showcasing the different conditions experienced by different parts of the province? 

In order to visualize the weather conditions on a map, I choose to use two versions of the data. The reason for doing this was because I wanted to use different color icons representing different primary conditions. First, I replace all the '[]' in the 'Secondary Conditions' column to NaN and then drop those rows. I plot the latitude and longitudes on the map and attach information to the associated coordinates such as [primary conditions, visibility, last updated]. If you zoom into the map, you will see different color of icons. These corresponds to the official definitions used by Alberta, see link https://511.alberta.ca/about/definitions. For example, the black icon means Bare (Dry/Wet) etc. For some reason I could not use yellow color for the Partly covered snow, so it shows up as gray. 

In [19]:
df_new = df.replace('[]',np.NaN)
df_cleaned = df_new[['LocationDescription', 'Primary Condition', 'Secondary Conditions',
       'Visibility', 'AreaName', 'RoadwayName','latitude','longitude','DateTime']].dropna()
df_cleaned.shape

(54, 9)

Since I have the above dataset with 'Secondary Conditions' != '[]'. The following dataset consists of just observations where 'Secondary conditons' == []' rows. I add these to the map and the color of the icon is set to 'red' meaning Closed/closures as per the definitions. 

In [20]:
df_closure = df[df['Secondary Conditions']=='[]']
df_closure.shape

(484, 9)

In [31]:
AB_COORDINATES = (53.933270, -116.576508)

# create empty map zoomed in on Alberta, location parameter takes latitudes and longitudes as starting location
map = folium.Map(location=AB_COORDINATES, zoom_start=5)

tooltip = 'Click me!'
mc = MarkerCluster()

# Function to change the marker color
# according to the elevation of volcano


def color(condition):
    if condition == 'Bare Wet' or condition == 'Bare Dry':
        col = 'black'
    elif condition == 'Ptly Cvd Snw' or condition == 'Ptly Cvd Ice':
        col = 'gray'
    elif condition == 'Cvd Snw' or condition == 'Cvd Ice':
        col = 'purple'
    else:
        col = 'red'
    return col
# def picture(condition):
#     if condition == 'Bare Wet' or condition == 'Bare Dry' :
#         pic = folium.features.CustomIcon('bare.png',icon_size=(30,30))
#     elif condition == 'Ptly Cvd Snw' or condition == 'Ptly Cvd Ice':
#         pic = folium.features.CustomIcon('partlycovered.png',icon_size=(10,10))
#     elif condition == 'Cvd Snw' or condition == 'Cvd Ice':
#         pic = folium.features.CustomIcon('covered.png',icon_size=(10,10))
#     else:
#         pic='red'
#     return pic


# add a marker for every record in the filtered data, use a clustered view
for lat, lan, pri, vis, t, sec, des in zip(df_cleaned['latitude'], df_cleaned['longitude'],
                                           df_cleaned['Primary Condition'], df_cleaned['Visibility'],
                                           df_cleaned['DateTime'], df_cleaned['Secondary Conditions'],
                                           df_cleaned['LocationDescription']):
    popup = folium.Popup('<strong> Primary Condition: </strong>' + pri + "," + ' <strong>Secondary Condition: </strong>' + sec
                         + ',<strong> Location description: </strong> ' +
                         des + ',<strong> Visibility: </strong>'
                         + vis + ',<strong> Last Updated: </strong>' + str(t), max_width=250)
    mc.add_child(folium.Marker(location=[lat, lan], popup=popup, icon=folium.Icon(color=color(pri), icon_color='yellow',
                                                                     icon='info-sign'))).add_to(map)

for lat,lan,t,desc in zip(df_closure['latitude'],df_closure['longitude'],df_closure['DateTime'], 
                         df_closure['LocationDescription']):
    popup = folium.Popup('<strong> Condition</strong>: Closed, ' +',<strong> Location description: </strong> '+ desc + '</strong>' 
                         + ', <strong>Last Updated: </strong>' + str(t), max_width=250)
    mc.add_child(folium.Marker(location=[lat,lan],popup = popup, icon= folium.Icon(color='red', icon_color='yellow',
                                                                                   icon='info-sign'))).add_to(map)
map.add_child(mc)
map

The map is interactive, the number on the circles are basically clusters of coordinates, for instance, 32 means that part of the map consists of 32 observations. Click on the cluster to zoom in. By clicking on the location icons, you can get more information. It would be interesting if the government introduces an app where such interactive map can be presented (probably exist I might not be aware of one). Also, can highlight the highways using the series of the coordinates which we saw earlier in the decoding process.  